### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import requests
import pageviewapi
import datetime as dt
from bs4 import BeautifulSoup
import altair as alt
import us

In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [4]:
import altair as alt
import altair_stiles as altstiles

alt.themes.register("stiles", altstiles.theme)
alt.themes.enable("stiles")

ThemeRegistry.enable('grid')

In [5]:
from datawrapper import Datawrapper

dw = Datawrapper(
    access_token="PU52xuCQowgE5BFprvdnpFXzd5Ql9ISOSLBk58eNR2ykPztw0yC5fpqLTfsZdhyR"
)

In [6]:
from oauth2client.service_account import ServiceAccountCredentials
import gspread

In [7]:
from df2gspread import df2gspread as d2g

### wrangle state data and put it in a google sheet to collect columbus day data

In [8]:
scope = ["https://spreadsheets.google.com/feeds"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    "jupyter-integration-Copy1.json", scope
)
gc = gspread.authorize(credentials)

In [9]:
spreadsheet_key = "1n39se9pfMExKKazI13FJEjIu9tjYYejJndcGb3MuonU"

In [10]:
all_features = []

for st in us.states.STATES:
    states_dict = {
        "fips": st.fips,
        "name": st.name,
        "abbr": st.abbr,
        "is_continental": st.is_continental,
        "statehood_year": st.statehood_year,
        "capital": st.capital,
        "capital_tz": st.capital_tz,
        "ap_abbr": st.ap_abbr,
        "shapefile_urls": st.shapefile_urls(),
    }
    all_features.append(states_dict)

In [11]:
df = (
    pd.DataFrame(all_features)
    .sort_values("fips", ascending=True)
    .reset_index(drop=True)
)

In [12]:
df = df[["fips", "name", "abbr", "ap_abbr", "capital"]]

In [13]:
df.head(10)

,fips,name,abbr,ap_abbr,capital
0,01,Alabama,AL,Ala.,Montgomery
1,02,Alaska,AK,Alaska,Juneau
2,04,Arizona,AZ,Ariz.,Phoenix
3,05,Arkansas,AR,Ark.,Little Rock
4,06,California,CA,Calif.,Sacramento
5,08,Colorado,CO,Colo.,Denver
6,09,Connecticut,CT,Conn.,Hartford
7,10,Delaware,DE,Del.,Dover
8,12,Florida,FL,Fla.,Tallahassee
9,13,Georgia,GA,Ga.,Atlanta


#### link to wikipedia page with timeline of indigenous peoples' day
#### [beautiful soup scraping guide](https://github.com/allanjamesvestal/teaching-guide-python-scraping)

In [16]:
url = "https://en.wikipedia.org/wiki/Indigenous_Peoples%27_Day"

In [17]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen(url)

In [18]:
soup = BeautifulSoup(html, "html.parser")

In [26]:
len(soup.find_all("span", class_="mw-headline"))

16

In [34]:
year_adopted = []
heds = soup.find_all("span", class_="mw-headline")[5:12]
for hed in heds:
    year_adopted.append(hed.text)

In [35]:
year_adopted

['Adopted 1989',
 'Adopted 2015',
 'Adopted 2016',
 'Adopted 2018',
 'Adopted 2019',
 'Adopted 2020',
 'Adopted 2021']

In [42]:
state_lists=soup.find_all("ul")[3:10]

[<ul><li><a href="/wiki/South_Dakota" title="South Dakota">South Dakota</a><sup class="reference" id="cite_ref-37"><a href="#cite_note-37">[37]</a></sup> (celebrated as Native American Day, instead of Columbus Day)</li></ul>,
 <ul><li><a href="/wiki/Alaska" title="Alaska">Alaska</a><sup class="reference" id="cite_ref-38"><a href="#cite_note-38">[38]</a></sup> (celebrated instead of Columbus Day)</li></ul>,
 <ul><li><a href="/wiki/Minnesota" title="Minnesota">Minnesota</a><sup class="reference" id="cite_ref-39"><a href="#cite_note-39">[39]</a></sup></li>
 <li><a href="/wiki/Vermont" title="Vermont">Vermont</a><sup class="reference" id="cite_ref-40"><a href="#cite_note-40">[40]</a></sup> (stopped celebrating Columbus Day in 2019)<sup class="reference" id="cite_ref-41"><a href="#cite_note-41">[41]</a></sup></li></ul>,
 <ul><li><a href="/wiki/Iowa" title="Iowa">Iowa</a><sup class="reference" id="cite_ref-42"><a href="#cite_note-42">[42]</a></sup></li>
 <li><a href="/wiki/North_Carolina" ti